## Introduction to Data Science

## The CRISP-DM approach to Data Science Tasks

_________________________

## I. Business Understanding
                    
#### The Business Understanding phase focuses on understanding the objectives and requirements of the project.  
+ **Determine business objectives**:   Construir um detector de fake news.
+ **Assess situation**:  A detecção de fake news já faz parte do subconsciente do usuário diário da internet. Felizmente podemos traduzir nossa intuição por trás de um título duvidoso e de palavras no imperativo para termos de Machine Learing. 
+ **Determine data mining goals** :  Buscar por sites de notícias que analisam veracidade de notícias, extraindo URLs, manchetes, subtítulos e outras informações pertinentes.
+ **Produce project plan** :  
Avaliar fontes de coleta de dados.
Selecionar critério para confiabilidade de um site.
Coletar e limpar os dados.
Construir e examinar modelo Tfidf.
Averiguar os resultados e reavaliar o modelo.
Revisar o projeto e realizar o deploy.





## II. Data Understanding  

+ Collect initial data:  
+ Describe data:  
+ Explore data:  
+ Verify data quality:  

### Metas!!!

+ Organizar categorias de notícias e os veículos de transmissão
+ Encontrar pessoas/instiuições/temas recorrentes
+ Atentar para caracteres Unicode
+ Determinar filtros (o que vamos armazenar do texto fonte).

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from selenium import webdriver

import requests
import urllib
from bs4 import BeautifulSoup as bs

%matplotlib inline

In [2]:
url = 'https://piaui.folha.uol.com.br/lupa/tag/verificamos/'
content = requests.get(url)
print(content.status_code)

200


In [2]:
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
options = webdriver.chrome.options.Options()
options.headless = True

driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)
driver.get("https://piaui.folha.uol.com.br/lupa/tag/verificamos/")



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.15/chromedriver_linux64.zip
Driver has been saved in cache [/home/isaque/.wdm/drivers/chromedriver/linux64/93.0.4577.15]


In [6]:
blocos = driver.find_elements_by_class_name('bloco')


In [7]:
import inspect

inspect.getmembers(blocos, lambda a:not(inspect.isroutine(a)))

[('__add__', <method-wrapper '__add__' of list object at 0x7f4865665a80>),
 ('__class__', list),
 ('__contains__',
  <method-wrapper '__contains__' of list object at 0x7f4865665a80>),
 ('__delattr__',
  <method-wrapper '__delattr__' of list object at 0x7f4865665a80>),
 ('__delitem__',
  <method-wrapper '__delitem__' of list object at 0x7f4865665a80>),
 ('__doc__',
  'Built-in mutable sequence.\n\nIf no argument is given, the constructor creates a new empty list.\nThe argument must be an iterable if specified.'),
 ('__eq__', <method-wrapper '__eq__' of list object at 0x7f4865665a80>),
 ('__ge__', <method-wrapper '__ge__' of list object at 0x7f4865665a80>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of list object at 0x7f4865665a80>),
 ('__gt__', <method-wrapper '__gt__' of list object at 0x7f4865665a80>),
 ('__hash__', None),
 ('__iadd__', <method-wrapper '__iadd__' of list object at 0x7f4865665a80>),
 ('__imul__', <method-wrapper '__imul__' of list object at 0x7f4865665

In [8]:
for i in blocos:
    print(i.text)

08.09.2021 | 17H58 | NO FACEBOOK
#Verificamos: Nota da Transparência Internacional no Brasil sobre inquérito das fake news é de 2019
Data da declaração foi apagada digitalmente, e documento tem sido exibido como se fosse atual; contexto também foi omitido
CAROL MACÁRIO
08.09.2021 | 11H12 | NO FACEBOOK
#Verificamos: É falso que Alexandre de Moraes fugiu do país após ameaças de Bolsonaro
Ministro está em Brasília e participa de sessão do STF nesta quarta (8)
GUSTAVO QUEIROZ
07.09.2021 | 12H32 | NO FACEBOOK
#Verificamos: É falso gráfico de ‘prisões arbitrárias’ de conservadores publicado pelo ex-ministro Weintraub
Imagem original, que ilustra um teste de controle de qualidade, foi tirada da Wikipédia e teve conteúdo manipulado, com legendas cortadas
BRUNO NOMURA
06.09.2021 | 17H08 | NO FACEBOOK
#Verificamos: É falso que empresa de laticínios Tirol tenha proibido uso de bandeiras do Brasil em suas instalações
Informação que circula nas redes sociais foi desmentida em comunicado
MARCELA DUA

In [14]:
print(blocos[0].find_element_by_tag_name("a").get_attribute("href"))

https://piaui.folha.uol.com.br/lupa/2021/09/08/verificamos-transparencia-internacional-brasil-inquerito-fake-news/


In [22]:
try:
    a = blocos[0].find_element_by_class_name("bloco-title")
except:
    print("not cool")
print(a)

<selenium.webdriver.remote.webelement.WebElement (session="596ba7120928816f2af74263ee2d1e35", element="70892646-640c-49cd-83c5-31dcbd268050")>


In [4]:
def extract_link(bloco):
    try:
        title = bloco.find_element_by_class_name("bloco-title")
    except:
        return -1
    
    return bloco.find_element_by_tag_name("a").get_attribute("href")

    

In [34]:
extract_link(blocos[1])

'https://piaui.folha.uol.com.br/lupa/2021/09/08/verificamos-alexandre-moraes-fugiu/'

In [37]:
link = extract_link(blocos[1])
crawler = webdriver.Chrome(ChromeDriverManager().install(), options = options)
crawler.get(link)
texto = crawler.find_element_by_class_name("wrapper")



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/home/isaque/.wdm/drivers/chromedriver/linux64/93.0.4577.15/chromedriver] found in cache


In [38]:
texto.text

'INÍCIO\nASSINE'

In [41]:
teste = crawler.find_element_by_class_name("etiqueta")

In [43]:
teste.text

'FALSO'

In [55]:
titulo = crawler.find_element_by_class_name("post-inner")
titulo.text

'Circula nas redes sociais que o ministro do Supremo Tribunal Federal (STF) Alexandre de Moraes estaria deixando o país nesta terça (7). O motivo seriam as manifestações de grupos bolsonaristas.  Por meio do projeto de verificação de notícias, usuários do Facebook solicitaram que esse material fosse analisado. Confira a seguir o trabalho de verificação da Lupa:\n“Moraes deve deixar o país na madrugada desta terça”\nPublicação que circula no Facebook e que, até as 11h do dia 8 de setembro de 2021, tinha sido compartilhada por mais de 250 pessoas\nFALSO\nAssine a Lente, a newsletter gratuita sobre desinformação da Lupa!\nA informação analisada pela Lupa é falsa. Segundo a assessoria de imprensa do Supremo Tribunal Federal (STF), o ministro Alexandre de Moraes não deixou o país na madrugada desta terça (7) e também não pretende viajar ao exterior nesta semana. A assessoria da corte afirmou, ainda, que o ministro estará em sessão presencial no Supremo hoje (8).\nA peça de desinformação dá 

# Prepare to scrape!

In [7]:
N = 10 #number of pages to visit
n = 10 #number of links per page
options = webdriver.chrome.options.Options()
options.headless = True

driver  = webdriver.Chrome(ChromeDriverManager().install(), options = options)
crawler = webdriver.Chrome(ChromeDriverManager().install(), options = options)


for flip in range(N):
    
    page = f'https://piaui.folha.uol.com.br/lupa/tag/verificamos/page/{flip+75}'
    
    driver.get(page)
    
    blocos = driver.find_elements_by_class_name('bloco')
    
    for b in range(n):
        
        link = extract_link(blocos[b])
        crawler.get(link)
        titulo = crawler.find_element_by_class_name("post-inner")
        string = titulo.text
        print(string)
        print()
        print(20*'-')
        print()





====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/home/isaque/.wdm/drivers/chromedriver/linux64/93.0.4577.15/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/home/isaque/.wdm/drivers/chromedriver/linux64/93.0.4577.15/chromedriver] found in cache


Circula nas redes sociais um vídeo que mostra dois carros-pipas descartando água irregularmente. A legenda que acompanha a gravação afirma que os veículos estavam pegando água de um canal inaugurado pelo presidente Jair Bolsonaro (sem partido) e jogando fora. Por meio do projeto de verificação de notícias, usuários do Facebook solicitaram que esse material fosse analisado. Confira a seguir o trabalho de verificação da Lupa:
“Pegando água no canal que foi inaugurado, pelo presidente e jogando fora. Essa é a resistência!! É muita maldade #BolsonaroTemRazao“
Legenda da imagem publicada no Facebook que, até às 13h do dia 09 de julho de 2020, tinha sido compartilhada por 200 pessoas 
FALSO
Assine a Lente, a newsletter gratuita sobre desinformação da Lupa!
O vídeo analisado pela Lupa é antigo. A gravação foi feita em junho de 2017 em Pariconha, município de Alagoas, e mostra dois caminhões-pipas do Exército fazendo o descarte irregular de água. O vídeo mostra que o líquido estaria sendo joga

Circulam nas redes sociais duas fotos de uma pessoa com toda a roupa manchada de sangue. A legenda diz que o homem que aparece nas imagens seria um motorista de ônibus, e foi ferido com um paralelepípedo após reclamar de um passageiro que estava sem máscara. Por meio do projeto de verificação de notícias, usuários do Facebook solicitaram que esse material fosse analisado. Confira a seguir o trabalho de verificação da Lupa:
“Esse motorista de ônibus acabou de levar uma pedrada na boca com um paralelepípedo e teve o maxicilar quebrado por um passageiro que queria subir no Coletivo sem máscara e ele reclamou, vamos compartilhar até que a justiça seja feita”
Legenda de imagem publicada no Facebook que, até as 18h do dia 08 de julho de 2020, tinha sido compartilhada por mais de 9,9 mil pessoas
FALSO
A informação analisada pela Lupa é falsa. A agressão ocorreu em outubro de 2015, no Recife (PE). A foto, publicada pelo Diário de Pernambuco, é de um motorista de ônibus que foi agredido por uma

Circula nas redes sociais que detentos que foram libertados provisoriamente por causa da pandemia de Covid-19 cometeram 13,9% dos homicídios no mês de abril. Por meio do projeto de verificação de notícias, usuários do Facebook solicitaram que esse material fosse analisado. Confira a seguir o trabalho de verificação da Lupa:
“Quer dizer que os presos soltos pelo Covid-19 já correspondem a 13,9% dos homicídios em abril? Alguém avisa aos iluminados governadores dos estados e ministros do Supremo, que os presos soltos pelo Covid-19 já matam mais do que o Covid-19. Espero ter ajudado!”
Imagem em post publicado no Facebook que, até as 14h de 8 de julho de 2020, tinha sido compartilhado por cerca de 500 pessoas
FALSO
Assine a Lente, a newsletter gratuita sobre desinformação da Lupa!
A informação analisada pela Lupa é falsa. Não existe nenhuma compilação estatística nacional sobre a autoria de homicídios, visto que, na maioria dos casos, o autor não é conhecido. O número citado nessa peça de d

Circulam pelas redes sociais duas imagens que mostram um grupo de banhistas em um canal da transposição do rio São Francisco. De acordo com a legenda, as fotos foram registradas recentemente, durante o governo de Jair Bolsonaro (sem partido). Em 26 de junho, o presidente inaugurou um trecho do Eixo Norte da obra, no Ceará. O texto diz que “agora, o sertão virou mar”, numa referência à profecia atribuída a Antônio Conselheiro (1830-1897), líder da revolta de Canudos, ocorrida na Bahia no final do século 19. Por meio do projeto de verificação de notícias, usuários do Facebook solicitaram que esse material fosse analisado. Confira a seguir o trabalho de verificação da Lupa:
“Veja, o ‘SERTÃO VIROU MAR.’
Pátria Amada Brasil – Bolsonaro 2022″
Assine a Lente, a newsletter gratuita sobre desinformação da Lupa!
Legenda de post publicado no Facebook que, até as 10h30 de 7 de julho de 2020, tinha 496 compartilhamentos
FALSO
A informação analisada pela Lupa é falsa. As fotos dos banhistas em um ca

...

Circula nas redes sociais que a jornalista Renata Vasconcellos testou positivo para a Covid-19, doença causada pelo novo coronavírus. Esse boato começou a circular pelas redes depois que ela não apresentou o Jornal Nacional, da Globo, na segunda-feira. Por meio do projeto de verificação de notícias, usuários do Facebook solicitaram que esse material fosse analisado. Confira a seguir o trabalho de verificação da Lupa:
“Renata Vasconcelos testou positivo para o covid 19. Acho que ela não deve tomar cloroquina, pois o JN é contra!!!! Vamos acompanhar!!!!”
Legenda de imagem que, até às 12h do dia 4 de junho de 2020, tinha sido compartilhado por mais 800 pessoas no Facebook
FALSO
Assine a Lente, a newsletter gratuita sobre desinformação da Lupa!
A informação analisada pela Lupa é falsa. Embora a jornalista Renata Vasconcellos não tenha apresentado o Jornal Nacional na segunda (1) e na terça-feira (2) desta semana, ela retornou na quarta (3). Na sua ausência, a jornalista Mariana Gross apres

Circula nas redes sociais um tuíte atribuído ao ministro interino da Saúde, general Eduardo Pazuello. O texto diz o seguinte: “A @policiafederal deveria estar atrás de corruptos, ladrões, traficantes, PCC. Deixem as pessoas de bem ter suas opiniões.! Isso é Democracia”. Por meio do projeto de verificação de notícias, usuários do Facebook solicitaram que esse material fosse analisado. Confira a seguir o trabalho de verificação da Lupa:
“A @policiafederal deveria estar atrás de corruptos, ladrões, traficantes, PCC. Deixem as pessoas de bem ter suas opiniões.! Isso é Democracia”
Tuíte atribuído ao general Eduardo Pazuello que, até às 12h do dia 3 de junho de 2020, tinha sido compartilhado por mais 500 pessoas
Assine a Lente, a newsletter gratuita sobre desinformação da Lupa!
FALSO
O tuíte analisado pela Lupa não foi publicado pelo ministro interino da Saúde, Eduardo Pazuello. Em seu site, o Ministério da Saúde informou que o ministro não tem perfis em redes sociais e que já solicitou à pl

Circula nas redes sociais uma imagem de vários sacos alinhados na rua e de uma pessoa carregando um dos sacos com apenas uma mão. A legenda que acompanha a fotografia indica que esses sacos seriam corpos de vítimas da Covid-19. “Inventam mortes, inventam enterros, inventam caixões vazios, inventam corpos de mentira, roubam o dinheiro do Covid, aterrorizam a população”, diz a legenda. Por meio do projeto de verificação de notícias, usuários do Facebook solicitaram que esse material fosse analisado. Confira a seguir o trabalho de verificação da Lupa:
“A safadeza é grande de mais. Inventam mortes, inventam enterros, inventam caixões vasios, inventam corpos de mentira, roubam o dinheiro do covid, aterrorizam a população. E depois joga a culpa no bolsonaro”
Legenda de imagem compartilhada no Facebook que, até às 17h do dia 2 de junho de 2020, tinha sido compartilhado por mais 700 pessoas
FALSO
Assine a Lente, a newsletter gratuita sobre desinformação da Lupa!
A fotografia analisada pela Lup

Circula pelas redes sociais uma imagem com uma frase atribuída ao ex-primeiro ministro do Reino Unido Winston Churchill (1874-1965). De acordo com a publicação, ele teria dito que “os fascistas do futuro chamarão a si mesmos de antifascistas”. Seria ao mesmo tempo uma previsão do surgimento do movimento Antifa e uma crítica a esse grupos, que se opõem a conservadores e à extrema-direita. Por meio do projeto de verificação de notícias, usuários do Facebook solicitaram que esse material fosse analisado. Confira a seguir o trabalho de verificação da Lupa:
“Os fascistas do futuro chamarão a si mesmos de antifascistas”
Frase atribuída ao ex-primeiro ministro do Reino Unido Winston Churchill em post publicado no Facebook que, até as 17h de 1º de junho de 2020, tinha 715 compartilhamentos
FALSO
A informação analisada pela Lupa é falsa. A Sociedade Internacional Churchill, uma organização criada para preservar a memória e o legado do ex-primeiro ministro britânico, afirmou, em nota, que ele nu

Circula pelas redes sociais uma foto com integrantes da Gaviões da Fiel, uma das torcidas organizadas do Corinthians, caminhando pela rua. De acordo com a legenda, as pessoas mostradas na imagem são manifestantes que a imprensa classificou como “novos defensores da democracia”. No último domingo (31), houve um protesto em defesa da democracia e contra o presidente Jair Bolsonaro (sem partido) na avenida Paulista, na capital paulista. O ato foi organizado por torcedores dos quatro principais clubes de futebol de São Paulo: Corinthians, Palmeiras, São Paulo e Santos. Por meio do projeto de verificação de notícias, usuários do Facebook solicitaram que esse material fosse analisado. Confira a seguir o trabalho de verificação da Lupa:
“Estes manifestantes (segundo a imprensa) são os novos defensores da DEMOCRACIA!”
Legenda de post publicado no Facebook que, até as 12h de 1º de junho de 2020, tinha mais de 1,2 mil compartilhamentos
FALSO
Assine a Lente, a newsletter gratuita sobre desinforma

## III. Data Preparation

+ Select data:   
+ Clean data:   
+ Construct data/transform data/feature engineering:   
+ Integrate data:   
+ (Re) Format data:   

## IV. Modeling

+ Select modeling techniques: 
+ Generate test design: 
+ Build model: 
+ Assess model:  

## V. Evaluation                    

+ Evaluate results: 
+ Review process: 
+ Determine next steps: 

## VI. Deployment

+ Plan deployment: 
+ Plan monitoring and maintenance: 
+ Produce final report: 
+ Review project: 